In [23]:
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd
sys.path.append('/Users/jonathan/wine_analysis_hplc_uv/')
sys.path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['/Users/jonathan/wine_analysis_hplc_uv/wine_deg_study',
 '/Users/jonathan/.pyenv/versions/3.11.1/lib/python311.zip',
 '/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11',
 '/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/lib-dynload',
 '',
 '/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/site-packages',
 '/Users/jonathan/wine_analysis_hplc_uv/',
 '/Users/jonathan/wine_analysis_hplc_uv/']

In [24]:
from google_sheets_api import post_df_as_sheet_values

path_to_certs = os.path.join(os.getcwd(), '..')
path_to_certs = os.path.join(path_to_certs, 'agilette/modules/credientals_tokens')
sheet_id = '15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY'
sheet_range = 'wine_deg_sequence_table!A1:H500'
print(path_to_certs)

/Users/jonathan/wine_analysis_hplc_uv/wine_deg_study/../agilette/modules/credientals_tokens


In [25]:
import datetime

start_time = datetime.datetime.now().replace(hour=9, minute = 0, second = 0, microsecond = 0) + datetime.timedelta(days=1)
end_time = start_time + datetime.timedelta(days=14, hours = 8)

time_list = []
while start_time < end_time:
    time_list.append(start_time)
    start_time += datetime.timedelta(hours = 1)

In [26]:
import pandas as pd

seq_table = pd.DataFrame()

ambient_wines = ['a0101', 'a0201', 'a0301']

seq_table['time'] = time_list
seq_table['weekday'] = seq_table['time'].dt.day_name()
seq_table['experiment'] = pd.Series(ambient_wines*9)
seq_table['rep_number'] = ""
seq_table['runner'] = ""
seq_table.head()

,time,weekday,experiment,rep_number,runner
0,2023-04-13 09:00:00,Thursday,a0101,,
1,2023-04-13 10:00:00,Thursday,a0201,,
2,2023-04-13 11:00:00,Thursday,a0301,,
3,2023-04-13 12:00:00,Thursday,a0101,,
4,2023-04-13 13:00:00,Thursday,a0201,,


Every Tuesday and Friday the instrument will be reserved for library runs.

In [27]:
seq_table.loc[(seq_table['time'].dt.time >= datetime.time(9,00,00)) & ((seq_table['weekday'] == 'Tuesday') |(seq_table['weekday'] == 'Friday')), 'experiment'] = 'library runs'
seq_table.loc[39:50,:]

,time,weekday,experiment,rep_number,runner
39,2023-04-15 00:00:00,Saturday,NaN,,
40,2023-04-15 01:00:00,Saturday,NaN,,
41,2023-04-15 02:00:00,Saturday,NaN,,
42,2023-04-15 03:00:00,Saturday,NaN,,
43,2023-04-15 04:00:00,Saturday,NaN,,
44,2023-04-15 05:00:00,Saturday,NaN,,
45,2023-04-15 06:00:00,Saturday,NaN,,
46,2023-04-15 07:00:00,Saturday,NaN,,
47,2023-04-15 08:00:00,Saturday,NaN,,
48,2023-04-15 09:00:00,Saturday,NaN,,


In [52]:
import numpy as np
def planning_df_genner():
    planning_df = pd.DataFrame()

    planning_df['weekdays'] = pd.date_range("2023-04-12", "2023-04-27")
    planning_df['day'] = planning_df['weekdays'].dt.day_name()
    planning_df['weekday_num'] = planning_df['weekdays'].dt.weekday

    planning_df.loc[((planning_df.weekdays == '2023-04-12') | (planning_df.weekdays.dt.day % 2 ==0)), 'amb_run'] = 'y'
    planning_df['amb_run_cumsum'] = planning_df['amb_run'].apply(lambda x : 1 if x == 'y' else 0)
    planning_df['amb_run_cumsum'] = planning_df['amb_run_cumsum'].cumsum()

    planning_df.loc[~(planning_df.weekdays.dt.date == datetime.date(2023,4,12)) & (planning_df.weekdays.dt.day % 2 != 0),'n_freezer_run'] = 'y'

    planning_df['n_freezer_cumsum'] = planning_df['n_freezer_run'].apply(lambda x : 1 if x == 'y' else 0)
    planning_df['n_freezer_cumsum'] = planning_df['n_freezer_cumsum'].cumsum()

    planning_df.loc[~(planning_df.weekdays.dt.date == datetime.date(2023,4,12)) & (planning_df.weekdays.dt.day % 2 != 0), 'e_freezer_run'] = 'y'

    planning_df['exp_day_num'] = np.arange(1,17)

    return planning_df

planning_df = planning_df_genner()
planning_df

,weekdays,day,weekday_num,amb_run,amb_run_cumsum,n_freezer_run,n_freezer_cumsum,e_freezer_run,exp_day_num
0,2023-04-12,Wednesday,2,y,1,NaN,0,NaN,1
1,2023-04-13,Thursday,3,NaN,1,y,1,y,2
2,2023-04-14,Friday,4,y,2,NaN,1,NaN,3
3,2023-04-15,Saturday,5,NaN,2,y,2,y,4
4,2023-04-16,Sunday,6,y,3,NaN,2,NaN,5
5,2023-04-17,Monday,0,NaN,3,y,3,y,6
6,2023-04-18,Tuesday,1,y,4,NaN,3,NaN,7
7,2023-04-19,Wednesday,2,NaN,4,y,4,y,8
8,2023-04-20,Thursday,3,y,5,NaN,4,NaN,9
9,2023-04-21,Friday,4,NaN,5,y,5,y,10


On a 'experiment' day, we'd have 3 ambient runs, 3 e_freezer runs and 3 n_freezer runs. Now the ambient can be sequenced, but the e_freezer and n_freezer would need to be done manually in order to account for ambient conditions post defrost. thus should do the freezer runs first.



So ambient runs will be done by Monday 2023-04-24, freezer runs done Wednesday 2023-04-26.

Technically I could schedule the ambient runs to go every day at 6am or something..

In [29]:
post_df_as_sheet_values(seq_table, sheet_id, sheet_range, path_to_certs)

{'spreadsheetId': '15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY',
 'updatedRange': 'wine_deg_sequence_table!A1:E345',
 'updatedRows': 345,
 'updatedColumns': 5,
 'updatedCells': 1725}

In [30]:
n_freezer_runs = ['n0101','n0201', 'n0301']


def sample_code_gen(exp = 'str', wines = int, rep = int):
    return [f"{exp}{str(wine+1).zfill(2)}{str(rep).zfill(2)}" for wine in range(0,wines)]

samples = sample_code_gen('n', 3, 1) + sample_code_gen('e', 3, 1)
samples

['n0101', 'n0201', 'n0301', 'e0101', 'e0201', 'e0301']

wines run from 1 to 3. reps run from 1 - 7 for n and e experiments, 3 for ambient.

In [31]:
freezer_run_sched = pd.DataFrame()

freezer_run_sched['time'] = pd.date_range(start = '09:00:00', end = '17:00:00', freq='H').strftime('%H:%M')
freezer_run_sched.loc[0:5,'samples'] = samples
freezer_run_sched

,time,samples
0,09:00,n0101
1,10:00,n0201
2,11:00,n0301
3,12:00,e0101
4,13:00,e0201
5,14:00,e0301
6,15:00,NaN
7,16:00,NaN
8,17:00,NaN


In [32]:
planning_df

,weekdays,day,weekday_num,amb_run,amb_run_cumsum,n_freezer_run,n_freezer_cumsum,e_freezer_run,exp_day_num
0,2023-04-12,Wednesday,2,y,1,NaN,0,NaN,1
1,2023-04-13,Thursday,3,NaN,1,y,1,y,2
2,2023-04-14,Friday,4,y,2,NaN,1,NaN,3
3,2023-04-15,Saturday,5,NaN,2,y,2,y,4
4,2023-04-16,Sunday,6,y,3,NaN,2,NaN,5
5,2023-04-17,Monday,0,NaN,3,y,3,y,6
6,2023-04-18,Tuesday,1,y,4,NaN,3,NaN,7
7,2023-04-19,Wednesday,2,NaN,4,y,4,y,8
8,2023-04-20,Thursday,3,y,5,NaN,4,NaN,9
9,2023-04-21,Friday,4,NaN,5,y,5,y,10


14 days, 9 hours a day. How many rows?

In [33]:
14*9

126

In [34]:
planning_df[planning_df['n_freezer_run'] == 'y']['weekdays'].shape

(8,)

In [35]:
126/8

15.75

In [36]:
weekdays = pd.concat([planning_df[planning_df['n_freezer_run'] == 'y']['weekdays']]*16).reset_index(drop = True)
weekdays.shape

(128,)

In [37]:
seq = freezer_run_sched['time']
hours = pd.concat([seq]*14, axis = 0).reset_index(drop = True)
hours

0      09:00
1      10:00
2      11:00
3      12:00
4      13:00
       ...  
121    13:00
122    14:00
123    15:00
124    16:00
125    17:00
Name: time, Length: 126, dtype: object

In [38]:

hour_schedule = pd.concat([weekdays, hours], axis = 1, ignore_index = True)[:-2]
hour_schedule.columns = ['date','hour']
hour_schedule

,date,hour
0,2023-04-13,09:00
1,2023-04-15,10:00
2,2023-04-17,11:00
3,2023-04-19,12:00
4,2023-04-21,13:00
...,...,...
121,2023-04-15,13:00
122,2023-04-17,14:00
123,2023-04-19,15:00
124,2023-04-21,16:00


In [39]:
planning_df

,weekdays,day,weekday_num,amb_run,amb_run_cumsum,n_freezer_run,n_freezer_cumsum,e_freezer_run,exp_day_num
0,2023-04-12,Wednesday,2,y,1,NaN,0,NaN,1
1,2023-04-13,Thursday,3,NaN,1,y,1,y,2
2,2023-04-14,Friday,4,y,2,NaN,1,NaN,3
3,2023-04-15,Saturday,5,NaN,2,y,2,y,4
4,2023-04-16,Sunday,6,y,3,NaN,2,NaN,5
5,2023-04-17,Monday,0,NaN,3,y,3,y,6
6,2023-04-18,Tuesday,1,y,4,NaN,3,NaN,7
7,2023-04-19,Wednesday,2,NaN,4,y,4,y,8
8,2023-04-20,Thursday,3,y,5,NaN,4,NaN,9
9,2023-04-21,Friday,4,NaN,5,y,5,y,10


7 normal and extreme wine repeats, 3 wines each = 21 individual elements.
n0101
n0102
n0201
etc.

In [40]:
sample_code_gen('n',3,7)

['n0107', 'n0207', 'n0307']

Need to generate blocks of nnneee then assign them to days corresponding to n_freezer_run from 9am to 3pm.

In [41]:
freezer_run_dates = planning_df.loc[planning_df['n_freezer_run']=='y','weekdays']
freezer_run_dates

1    2023-04-13
3    2023-04-15
5    2023-04-17
7    2023-04-19
9    2023-04-21
11   2023-04-23
13   2023-04-25
15   2023-04-27
Name: weekdays, dtype: datetime64[ns]

In [42]:
def sample_code_gen(exp = [], num_wines = int, num_rep = int):
    exp = pd.DataFrame(exp, columns = ['exp'])
    wine = pd.DataFrame([str(wine+1).zfill(2) for wine in range(0,num_wines)], columns = ['wine'])
    reps = pd.DataFrame([str(rep+1).zfill(2) for rep in range(0,num_rep)], columns =['rep'])
    
    exp['key'] = 1
    reps['key'] = 1
    wine['key'] = 1

    result = pd.merge(exp, reps, on = 'key')
    result = pd.merge(result, wine, on = 'key')[['exp','rep','wine']]

    result['code'] = result['exp'] + result['wine'] + result['rep']
    return result

codes = sample_code_gen(['a','n','e'],3,7)
codes.head()

,exp,rep,wine,code
0,a,01,01,a0101
1,a,01,02,a0201
2,a,01,03,a0301
3,a,02,01,a0102
4,a,02,02,a0202


In [43]:
codes[(codes.exp.isin(['n','e'])) & (codes['rep'].astype('int')==1)]

,exp,rep,wine,code
21,n,01,01,n0101
22,n,01,02,n0201
23,n,01,03,n0301
42,e,01,01,e0101
43,e,01,02,e0201
44,e,01,03,e0301


In [44]:
# hour_schedule.loc[hour_schedule['date'].isin(freezer_run_dates), 'exp'] = 

In [45]:
planning_df

,weekdays,day,weekday_num,amb_run,amb_run_cumsum,n_freezer_run,n_freezer_cumsum,e_freezer_run,exp_day_num
0,2023-04-12,Wednesday,2,y,1,NaN,0,NaN,1
1,2023-04-13,Thursday,3,NaN,1,y,1,y,2
2,2023-04-14,Friday,4,y,2,NaN,1,NaN,3
3,2023-04-15,Saturday,5,NaN,2,y,2,y,4
4,2023-04-16,Sunday,6,y,3,NaN,2,NaN,5
5,2023-04-17,Monday,0,NaN,3,y,3,y,6
6,2023-04-18,Tuesday,1,y,4,NaN,3,NaN,7
7,2023-04-19,Wednesday,2,NaN,4,y,4,y,8
8,2023-04-20,Thursday,3,y,5,NaN,4,NaN,9
9,2023-04-21,Friday,4,NaN,5,y,5,y,10


In [50]:
hour_schedule.sort_values(['date','hour'])

,date,hour,day_num,day
0,2023-04-13,09:00,3,Thursday
72,2023-04-13,09:00,3,Thursday
64,2023-04-13,10:00,3,Thursday
56,2023-04-13,11:00,3,Thursday
48,2023-04-13,12:00,3,Thursday
...,...,...,...,...
15,2023-04-27,15:00,3,Thursday
87,2023-04-27,15:00,3,Thursday
7,2023-04-27,16:00,3,Thursday
79,2023-04-27,16:00,3,Thursday


In [47]:
hour_schedule['day_num'] = hour_schedule['date'].dt.weekday
hour_schedule['day'] = hour_schedule['date'].dt.day_name()
hour_schedule

,date,hour,day_num,day
0,2023-04-13,09:00,3,Thursday
1,2023-04-15,10:00,5,Saturday
2,2023-04-17,11:00,0,Monday
3,2023-04-19,12:00,2,Wednesday
4,2023-04-21,13:00,4,Friday
...,...,...,...,...
121,2023-04-15,13:00,5,Saturday
122,2023-04-17,14:00,0,Monday
123,2023-04-19,15:00,2,Wednesday
124,2023-04-21,16:00,4,Friday
